In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
MONTHS_GEN = {
    1: "января",
    2: "февраля",
    3: "марта",
    4: "апреля",
    5: "мая",
    6: "июня",
    7: "июля",
    8: "августа",
    9: "сентября",
    10: "октября",
    11: "ноября",
    12: "декабря"
}

In [57]:
data = pd.read_csv('weather_data_3.csv', sep=',').drop(['text'], axis='columns')

In [58]:
data.head()

,13-14апреля,1мая,14декабря,9сентября,27мая,27-28мая,28января,2-3сентября,3ноября,17-18сентября,...,2декабря,28декабря,7марта,16февраля,8-9ноября,6января,4сентября,month,day_start,day_end
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,9,9
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,10,10
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,9,9
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,8
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,9,9


In [4]:
data = data.drop(['day_start','day_end'], axis='columns')

In [5]:
data.head()

,13-14апреля,1мая,14декабря,9сентября,27мая,27-28мая,28января,2-3сентября,3ноября,17-18сентября,...,25-26декабря,24июля,2декабря,28декабря,7марта,16февраля,8-9ноября,6января,4сентября,month
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8


In [59]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [75]:
X= data.drop('month', axis=1)
y = data['month'].astype('int')

In [76]:
X.shape, y.shape

((11874, 734), (11874,))

In [81]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [82]:
X_train.shape, X_valid.shape

((8311, 734), (3563, 734))

In [83]:
first_tree = DecisionTreeClassifier(random_state=17)

In [84]:
np.mean(cross_val_score(first_tree, X_train, y_train, cv=5))

0.9583678316122184

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, X_train, y_train, cv=5))

0.946215175190473

In [85]:
from sklearn.model_selection import GridSearchCV

In [107]:
tree_params = {'max_depth': np.arange(460, 481)}

In [108]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [109]:
%%time
tree_grid.fit(X_train, y_train);

Wall time: 1min 12s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': array([460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472,
       473, 474, 475, 476, 477, 478, 479, 480])})

In [110]:
tree_grid.best_score_, tree_grid.best_params_

(0.9587288424425433, {'max_depth': 480})

In [37]:
tree_valid_pred = tree_grid.predict(m_x_valid)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(m_y_valid, tree_valid_pred)

0.9994386752736458

In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(m_y_valid, tree_valid_pred)

array([[283,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 276,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 255,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 265,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 311,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 305,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 344,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 350,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 296,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 296,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 309,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 271]],
      dtype=int64)

In [42]:
from sklearn.tree import export_graphviz

In [43]:
second_tree = DecisionTreeClassifier(max_depth=12).fit(m_x_train, m_y_train)
second_tree.score(m_x_valid, m_y_valid)


0.9991580129104687

In [46]:
from utils import sentence_to_bagofwords

In [57]:
test_text_list = []
test_text_list.append('Желтый уровень опасности. 3 сентября (четверг) '
                      'местами в центральных и восточных районах страны'
                      ' ожидаются грозы.')
test_text_list.append('Желтый уровень опасности. 22 августа (суббота) в ночные'
                      ' и утренние часы местами по республике ожидается туман.'
                      ' В дневные часы в отдельных районах Брестской области'
                      ' максимальная температура воздуха достигнет +30&#176;С.')
test_text_list.append('Желтый уровень опасности. 27 июля (понедельник) в'
                      ' отдельных районах ожидаются грозы и сильные дожди,'
                      ' днем местами по Гомельской области максимальная'
                      ' температура воздуха достигнет +30.')
print(test_text_list)

['Желтый уровень опасности. 3 сентября (четверг) местами в центральных и восточных районах страны ожидаются грозы.', 'Желтый уровень опасности. 22 августа (суббота) в ночные и утренние часы местами по республике ожидается туман. В дневные часы в отдельных районах Брестской области максимальная температура воздуха достигнет +30&#176;С.', 'Желтый уровень опасности. 27 июля (понедельник) в отдельных районах ожидаются грозы и сильные дожди, днем местами по Гомельской области максимальная температура воздуха достигнет +30.']


In [60]:
test_bag_list = [sentence_to_bagofwords(i) for i in test_text_list]
print(sentence_to_bagofwords(test_text_list[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [61]:
for item in test_bag_list:
    print(item)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [62]:
test_arr = np.array(test_bag_list)

In [64]:
second_tree.predict(test_arr)

array([9, 8, 7])